In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions_labor_distribution

# Step 1: Get file path of report

In [2]:
#Pick the one from the "Export to SQL" folder
#report_path = report_lst[report_lst['ReportName'] == 'Locations']['ReportPath'].loc[52]
report_path = helper_functions_labor_distribution.get_report_file_path('Locations')

report_path 

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Locations']"

## Step 2: Retrieve the report and export as a CSV

In [3]:
df = helper_functions_labor_distribution.retrieve_report(report_path) 
df 

,location_code,location,inactivation_date,inactive
0,000000,Z-DO NOT USE,2016-01-01T00:00:00,Y
1,000100,CSO,'@xs:nil': 'true',N
2,000101,CSOJV - OSC Managers,'@xs:nil': 'true',N
3,000104,CSO - TN Finance,'@xs:nil': 'true',N
4,000105,CSO - Finance,'@xs:nil': 'true',N
...,...,...,...,...
548,AC0199,CSO - Ops Leadership,'@xs:nil': 'true',N
549,AD0182,CSO - Credit Resolution,'@xs:nil': 'true',N
550,AD0183,CSO - Central Bus Svs,'@xs:nil': 'true',N
551,AD0196,CSO - IT,'@xs:nil': 'true',N


In [4]:
#Remove the null values so that they say 0 instead
df['location_code'] = df['location_code'].astype(str).replace("'@xs:nil': 'true'", '0')

# Step 3: Export the report as a CSV

In [5]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [6]:
helper_functions_labor_distribution.create_sql_table_v2('ZZ_TEST_Location')

# Step 5: Close the API connection

In [7]:
helper_functions_labor_distribution.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'

Code to create the table: 

CREATE TABLE ZZ_TEST_Location(
	Location_Code NVARCHAR(255),
	Location NVARCHAR(255),
	Inactivation_Date NVARCHAR(255),
	Inactive NVARCHAR(255))